In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from io import StringIO

In [2]:
dfs_by_date = {}

for year in range(2018, 2025):
    for month in range(1, 13):
        year_str = str(year)
        month_str = f'{month:02d}'
        filename = f'./01_raw_data/vechicle/{year_str}년_{month_str}월_자동차_등록자료_통계.xlsx'
        
        try:
            df = pd.read_excel(filename, sheet_name='10.연료별_등록현황', header=None)
            dfs_by_date[(year, month)] = df  # 연도/월별로 저장
            print(f'{filename} 불러오기 성공')
        except FileNotFoundError:
            print(f'{filename} 파일이 존재하지 않음')
        except Exception as e:
            print(f'{filename} 오류 발생: {e}')

./01_raw_data/vechicle/2018년_01월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_02월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_03월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_04월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_05월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_06월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_07월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_08월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_09월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_10월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_11월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2018년_12월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2019년_01월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2019년_02월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2019년_03월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw_data/vechicle/2019년_04월_자동차_등록자료_통계.xlsx 파일이 존재하지 않음
./01_raw

In [3]:
df_list = {}

def preprocess(df_ori, list):
    df = df_ori.copy()
    ym = df.iloc[1, 1]
    year, month = ym.split('.')
    df.insert(0, '월', month)
    df.insert(0, '년도', year)
    df = df.drop(index=range(0, 3), axis=0)
    df.columns = ['년도', '월','연료', '차종', '용도', '서울','부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주', '전국']
    df[['연료','차종']] = df[['연료', '차종']].fillna(method='ffill')
    df.dropna(subset=df.columns[5:], how='all', inplace=True) 
    df = df.reset_index(drop=True)
    list[(int(year), int(month))] = df 
    return df
    

In [4]:
df_full = pd.DataFrame()  # 초기화

for year in range(2019, 2025):
    for month in range(1, 13):
        if (year, month) in dfs_by_date:
            df_temp = dfs_by_date[(year, month)]
            df_temp = preprocess(df_temp, df_list)
            for region in ['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
                            '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주','전국']:
                    df_region = df_temp[['년도', '월', region, '연료', '차종', '용도']].copy()
                    df_region = df_region.rename(columns={region: '등록수'})
                    df_region['지역'] = region 
                    
                    df_full = pd.concat([df_full, df_region], ignore_index=True)
                    
df_full = df_full[['년도', '월', '지역', '연료', '차종', '용도', '등록수']]
df_full = df_full.reset_index(drop=True) 

C:\Users\junjo\AppData\Local\Temp\ipykernel_126920\4162053303.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['연료','차종']] = df[['연료', '차종']].fillna(method='ffill')
C:\Users\junjo\AppData\Local\Temp\ipykernel_126920\4162053303.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['연료','차종']] = df[['연료', '차종']].fillna(method='ffill')
C:\Users\junjo\AppData\Local\Temp\ipykernel_126920\4162053303.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[['연료','차종']] = df[['연료', '차종']].fillna(method='ffill')
C:\Users\junjo\AppData\Local\Temp\ipykernel_126920\4162053303.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


In [5]:
df_full['date'] = pd.to_datetime(df_full['년도'].astype(str) + '-' +df_full['월'].astype(str) + '-01')

df_full = df_full.sort_values(['지역', '연료', '차종', '용도', 'date'])

df_full['순증가량'] = (
    df_full
    .groupby(['지역', '연료', '차종', '용도'])['등록수']
    .diff()
    .fillna(0)
    .astype(int)
)

df_full = df_full.sort_values(['년도', '월'], ascending=[True, True]).reset_index(drop=True)
df_full = df_full[df_full['date'] >= '2020-01-01']
df_full = df_full.drop(columns=['date'])
df_full.to_csv('./02_interim_data/vechicle_full_data.csv', index=False, encoding='utf-8-sig')

C:\Users\junjo\AppData\Local\Temp\ipykernel_126920\3318590337.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)


In [6]:
df_ev = df_full[df_full['연료'] == '전기']
df_total = df_full[df_full['연료'] == '총계']
df_hydrogen = df_full[df_full['연료'] == '수소']
df_ev = df_ev.reset_index(drop=True) 
df_total = df_total.reset_index(drop=True)
df_hydrogen = df_total.reset_index(drop=True) 
df_ev.to_csv('./02_interim_data/vechicle_ev_data.csv', index=False, encoding='utf-8-sig')
df_total.to_csv('./02_interim_data/vechicle_total_data.csv', index=False, encoding='utf-8-sig')
df_hydrogen.to_csv('./02_interim_data/vechicle_hydrogen_data.csv', index=False, encoding='utf-8-sig')


In [10]:
# 하이브리드만 뽑아서 연료명 통일
hyb = df_full[df_full['연료'].str.contains('하이브리드', na=False)].copy()
hyb['연료'] = '하이브리드'

# 등록수와 순증가량 둘 다 합치기
hybrid_agg = (
    hyb
    .groupby(
        ['년도','월','지역','차종','용도','연료'],
        as_index=False
    )
    # 여기서 등록수와 순증가량을 모두 sum
    [['등록수','순증가량']]
    .sum()
)

# 나머지 비하이브리드는 그대로
others = df_full[~df_full['연료'].str.contains('하이브리드', na=False)]

# 합치고
df_temp_hyb = pd.concat([others, hybrid_agg], ignore_index=True)

# 하이브리드만 골라내서
df_hyb = df_temp_hyb[df_temp_hyb['연료']=='하이브리드'].reset_index(drop=True)
df_hyb.to_csv('./02_interim_data/vechicle_hyb_data.csv', index=False, encoding='utf-8-sig')
